In [1]:
using Oscar

  ___   ___   ___    _    ____
 / _ \ / __\ / __\  / \  |  _ \  | Combining and extending ANTIC, GAP,
| |_| |\__ \| |__  / ^ \ |  ´ /  | Polymake and Singular
 \___/ \___/ \___//_/ \_\|_|\_\  | Type "?Oscar" for more information
o--------o-----o-----o--------o  | Documentation: https://docs.oscar-system.org
  S Y M B O L I C   T O O L S    | Version 1.7.0


In [2]:
function identify_parameters(M::GraphicalModel, return_ideal = false)

  # get graph data from the model
  G = graph(M)
  if typeof(G) === Oscar.MixedGraph
    E = edges(G, Directed)
  else
    E = edges(G)
  end
  
  # get rings and parametrization
  S, s = model_ring(M)
  R, t = parameter_ring(M)
  phi = parametrization(M)
  
  # setup the elimination ideal
  elim_ring, elim_gens = polynomial_ring(coefficient_ring(R), vcat(symbols(R), symbols(S)))
  inject_R = hom(R, elim_ring, elim_gens[1:ngens(R)])
  inject_S = hom(S, elim_ring, elim_gens[ngens(R) + 1:ngens(elim_ring)])
  I = ideal([inject_S(s)*inject_R(denominator(phi(s))) - inject_R(numerator(phi(s))) for s in gens(S)])
  
  # eliminate
  out = Dict()
  for e in E
    l = t[e]
    other_params = [inject_R(i) for i in values(t) if i != l]
    J = eliminate(I, other_params)
    if return_ideal
      out[e] = J
    else
      out[e] = contains_linear_poly(J, inject_R(l), ngens(R))
    end
  end
  
  return out
end

function print_identification(out)
    for edge in keys(out) 
      println(edge) 
      show(stdout, "text/plain", out[edge]) 
      println()
    end
end

print_identification (generic function with 1 method)

In [3]:
# acyclic directed mixed graph (ADMG) 1
D = [[1, 3], [2, 3], [3, 4]];
B = [[1, 3], [3, 4]];
G = graph_from_edges(Mixed, D, B);
M = gaussian_graphical_model(G);

# return the elimination ideal of edge 1->2
out = identify_parameters(M, true);
for edge in keys(out) 
  println(edge) 
  show(stdout, "text/plain", out[edge]) 
  println()
end

Edge(3, 4)
Ideal generated by
  s[1, 2]
  -s[1, 3]*s[2, 4] + s[1, 4]*s[2, 3]
  l[3, 4]*s[2, 3] - s[2, 4]
  l[3, 4]*s[1, 3] - s[1, 4]
Edge(1, 3)
Ideal generated by
  s[1, 2]
  -s[1, 3]*s[2, 4] + s[1, 4]*s[2, 3]
Edge(2, 3)
Ideal generated by
  s[1, 2]
  -s[1, 3]*s[2, 4] + s[1, 4]*s[2, 3]
  l[2, 3]*s[2, 2] - s[2, 3]


In [4]:
function contains_linear_poly(J, l, nparams)

  R = base_ring(J)
  
  # find the position of l in gens(R) and can check the corresponding entry of the exponent vector for linearity
  l_pos = findfirst(==(l), gens(R))
  
  for f in gens(J)
    exps = exponents(f)
    for v in exps
      if v[l_pos] == 1 && sum(v[1:nparams]) == 1
        return (true, f)
      end
    end
  end
  
  return (false, R(0))
end

contains_linear_poly (generic function with 1 method)

In [5]:
# acyclic directed mixed graph (ADMG)
D = [[1, 3], [2, 3], [3, 4]];
B = [[1, 3], [3, 4]];
G = graph_from_edges(Mixed, D, B);
M = gaussian_graphical_model(G);
println(identify_parameters(M))
println()
print_identification(identify_parameters(M))

# directed acyclic graph (DAG)
G = graph_from_edges(Directed, [[1, 2], [2, 3]]);
M = gaussian_graphical_model(G);
println(identify_parameters(M))
println()

# undirected graph
G = graph_from_edges(Undirected, [[1, 2], [2, 3]]);
M = gaussian_graphical_model(G);
println(identify_parameters(M, true)) 
println()

# comment on cyclic graphs -> future version of Oscar. Conceptually equal

Dict{Any, Any}(Edge(3, 4) => (true, l[3, 4]*s[2, 3] - s[2, 4]), Edge(1, 3) => (false, 0), Edge(2, 3) => (true, l[2, 3]*s[2, 2] - s[2, 3]))

Edge(3, 4)
(true, l[3, 4]*s[2, 3] - s[2, 4])
Edge(1, 3)
(false, 0)
Edge(2, 3)
(true, l[2, 3]*s[2, 2] - s[2, 3])
Dict{Any, Any}(Edge(1, 2) => (true, l[1, 2]*s[1, 1] - s[1, 2]), Edge(2, 3) => (true, l[2, 3]*s[2, 2] - s[2, 3]))

Dict{Any, Any}(Edge(3, 2) => Ideal (0), Edge(2, 1) => Ideal (0))



In [8]:
import Oscar: parameter_ring, GraphDict, GraphMap

const ColoredGGM{Undirected} = GaussianGraphicalModel{
  Graph{Undirected}, @NamedTuple{color::GraphMap{Undirected}}
}

@attr Tuple{
  QQMPolyRing,
  GraphDict{QQMPolyRingElem}
} function parameter_ring(GM::ColoredGGM{Undirected})
  G = graph(GM)
  colors = unique([
    [G.color[e] for e in edges(G)]; [G.color[v] for v in vertices(G)]
  ])
  R, x = polynomial_ring(QQ, varnames(GM)[:k] => colors)
  color_dict = Dict{String, MPolyRingElem}(
    color => x[i] for (i, color) in enumerate(colors))

  gens_dict = GraphDict{QQMPolyRingElem}(
    Dict{Union{Int, Edge}, QQMPolyRingElem}(merge(
      Dict(e => color_dict[G.color[e]] for e in edges(G)),
      Dict(v => color_dict[G.color[v]] for v in vertices(G))
  )))
    
  return R, gens_dict
end

parameter_ring (generic function with 19 methods)

In [21]:
G = graph_from_labeled_edges(Dict((1, 2) => "2, 1", (2, 3) => "3, 2"),
                                    Dict(1 => "red", 2 => "red", 3 => "red"); 
                             name=:color)


G = graph_from_edges([(1, 2), (2, 3)])
label!(G, Dict((1, 2) => "2, 1", (2, 3) => "3, 2"), Dict(1 => "red", 2 => "red", 3 => "red"); name=:color)



M = gaussian_graphical_model(G)
K = concentration_matrix(M)
show(stdout, "text/plain", K); println(); println()

out = identify_parameters(M)
print_identification(out)


[ k[red]   k[2, 1]         0]
[k[2, 1]    k[red]   k[3, 2]]
[      0   k[3, 2]    k[red]]



LoadError: UndefVarError: `print_dictionary` not defined in `Main`
Suggestion: check for spelling errors or missing imports.